<a href="https://colab.research.google.com/github/piabona/Dacon-Kaggle/blob/main/JEJU/%EC%A0%9C%EC%A3%BC_%ED%8A%B9%EC%82%B0%EB%AC%BC_ML04_Autogluon_(679_2711243475).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import random
from tqdm.auto import tqdm
# import optuna

DATA_PATH = '/content/drive/MyDrive/데이터 분석/데이콘 캐글 컴페티션/예슬 개인 참가/제주 특산물 가격 예측/data/'
# DATA_PATH = "C:\python-code\DACON_JEJU\data/"
SEED = 42

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

In [ ]:
import numpy as snp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import tree
# from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer

```
item : 품목 코드
TG : 감귤
BC : 브로콜리
RD : 무
CR : 당근
CB : 양배추
corporation : 유통 법인 코드
법인 A부터 F 존재
location : 지역 코드
J : 제주도 제주시
S : 제주도 서귀포시
supply(kg) : 유통된 물량, kg 단위
price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위
```

# DATA Load

In [ ]:
train_df = pd.read_csv(f"{DATA_PATH}train.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")
international = pd.read_csv(f"{DATA_PATH}international_trade.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [ ]:
train_df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'}, inplace=True)

In [ ]:
train_df['item_id'] = train_df.ID.str[0:6]
test_df['item_id'] = test_df.ID.str[0:6]


# Preprocessing

In [ ]:
# 공휴일
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

In [ ]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['year'] = train_df['timestamp'].dt.year
train_df['month_ori'] = train_df['timestamp'].dt.month
train_df['week'] = train_df['timestamp'].dt.isocalendar().week
train_df['day'] = train_df['timestamp'].dt.day
train_df['day_of_week'] = train_df['timestamp'].dt.dayofweek
# train_df['day_of_year'] = train_df['timestamp'].dt.dayofyear

train_df['sin_week'] = np.sin(2 * np.pi * train_df['week'] / 4)
train_df['cos_week'] = np.cos(2 * np.pi * train_df['week'] / 4)
train_df['sin_dayofweek'] = np.sin(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['cos_dayofweek'] = np.cos(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['holiday'] = np.where((train_df.day_of_week >= 5) | (train_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)


test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df['year'] = test_df['timestamp'].dt.year
test_df['month_ori'] = test_df['timestamp'].dt.month
test_df['week'] = test_df['timestamp'].dt.isocalendar().week
test_df['day'] = test_df['timestamp'].dt.day
test_df['day_of_week'] = test_df['timestamp'].dt.dayofweek
# test_df['day_of_year'] = test_df['timestamp'].dt.dayofyear

test_df['sin_week'] = np.sin(2 * np.pi * test_df['week'] / 4)
test_df['cos_week'] = np.cos(2 * np.pi * test_df['week'] / 4)
test_df['sin_dayofweek'] = np.sin(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['cos_dayofweek'] = np.cos(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['holiday'] = np.where((test_df.day_of_week >= 5) | (test_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [ ]:

import math

# 연중일 주기
def cos_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.cos(2 * math.pi * day_of_year / period)

def sin_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.sin(2 * math.pi * day_of_year / period)

train_df['cos_year'] = train_df['timestamp'].apply(cos_year)
train_df['sin_year'] = train_df['timestamp'].apply(sin_year)

test_df['cos_year'] = test_df['timestamp'].apply(cos_year)
test_df['sin_year'] = test_df['timestamp'].apply(sin_year)

# 요일 주기 (주간)
def cos_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.cos(2 * math.pi * day_of_week / period)

def sin_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.sin(2 * math.pi * day_of_week / period)

train_df['cos_dow'] = train_df['timestamp'].apply(cos_weekday)
train_df['sin_dow'] = train_df['timestamp'].apply(sin_weekday)

test_df['cos_dow'] = test_df['timestamp'].apply(cos_weekday)
test_df['sin_dow'] = test_df['timestamp'].apply(sin_weekday)

# 연중주 주기
def cos_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.cos(2 * math.pi * week_of_year / period)

def sin_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.sin(2 * math.pi * week_of_year / period)

train_df['cos_week_of_year'] = train_df['timestamp'].apply(cos_week_of_year)
train_df['sin_week_of_year'] = train_df['timestamp'].apply(sin_week_of_year)

test_df['cos_week_of_year'] = test_df['timestamp'].apply(cos_week_of_year)
test_df['sin_week_of_year'] = test_df['timestamp'].apply(sin_week_of_year)

## 월별 품목의 가격 추가(mean, max)

In [ ]:
train_df['month_of_year'] = train_df['timestamp'].dt.to_period('M')
test_df['month_of_year'] = test_df['timestamp'].dt.to_period('M')

In [ ]:
train_df.corr()

,supply,price,year,month_ori,week,day,day_of_week,sin_week,cos_week,sin_dayofweek,cos_dayofweek,holiday,cos_year,sin_year,cos_dow,sin_dow,cos_week_of_year,sin_week_of_year
supply,1.000000,-0.002297,0.024278,-0.120580,-0.122755,0.000161,-6.979763e-02,-8.460227e-04,-2.602730e-04,1.290241e-02,-5.266602e-02,-7.511494e-02,0.128117,0.127563,-2.274042e-02,4.921641e-02,0.120775,0.134423
price,-0.002297,1.000000,0.029442,-0.076673,-0.080900,0.011755,-1.526107e-01,-6.747611e-03,1.915609e-03,1.269470e-02,-1.413540e-01,-1.720389e-01,-0.087383,0.114127,-7.818649e-02,1.184160e-01,-0.094503,0.108096
year,0.024278,0.029442,1.000000,-0.114541,-0.110539,-0.010426,6.026246e-04,7.999659e-03,2.327758e-03,-9.028492e-04,5.628859e-04,-5.777979e-04,0.092760,0.057037,-3.551015e-04,-1.003063e-03,0.090264,0.060276
month_ori,-0.120580,-0.076673,-0.114541,1.000000,0.968650,0.014989,-2.460519e-03,-3.772849e-02,3.071251e-02,-1.706813e-05,-3.113063e-03,-1.493764e-02,-0.059733,-0.775603,-1.953875e-03,2.422900e-03,-0.020738,-0.772150
week,-0.122755,-0.080900,-0.110539,0.968650,1.000000,0.067706,2.038724e-04,-3.756553e-02,4.580337e-02,3.775076e-04,2.576626e-03,-7.643225e-03,-0.044620,-0.773588,1.901329e-03,-1.778775e-03,-0.005752,-0.770214
day,0.000161,0.011755,-0.010426,0.014989,0.067706,1.000000,8.848224e-05,-3.915163e-02,6.784983e-02,3.581593e-03,9.508420e-03,-2.646231e-02,-0.012314,-0.009078,8.727676e-03,-5.199336e-03,-0.011815,-0.009459
day_of_week,-0.069798,-0.152611,0.000603,-0.002461,0.000204,0.000088,1.000000e+00,-9.193557e-04,-1.389896e-18,-7.342618e-01,3.535005e-01,7.368009e-01,-0.000667,-0.000384,-3.537962e-01,-7.342457e-01,0.000335,-0.001886
sin_week,-0.000846,-0.006748,0.008000,-0.037728,-0.037566,-0.039152,-9.193557e-04,1.000000e+00,2.873764e-16,-4.469281e-20,-2.919584e-03,2.501863e-02,0.012490,0.001898,-1.819923e-03,2.282297e-03,0.012228,0.002654
cos_week,-0.000260,0.001916,0.002328,0.030713,0.045803,0.067850,-1.389896e-18,2.873764e-16,1.000000e+00,-1.102494e-18,-2.133724e-18,4.117079e-17,-0.002278,0.003339,7.049176e-18,2.047317e-18,-0.002436,0.003586
sin_dayofweek,0.012902,0.012695,-0.000903,-0.000017,0.000378,0.003582,-7.342618e-01,-4.469281e-20,-1.102494e-18,1.000000e+00,-2.227666e-17,-3.259428e-01,0.000199,-0.000319,7.819429e-01,6.236295e-01,-0.000538,0.000792


In [ ]:
item_month_price = train_df.groupby(['item_id', 'month_of_year'])
agg = {'price' : ['mean', 'max']}
price_df = item_month_price.agg(agg).reset_index()

In [ ]:
price_df.columns = ['item_id', 'month_of_year', 'month_price_mean', 'month_price_max']

train_df = pd.merge(train_df, price_df, on=['item_id', 'month_of_year'], how='left')
test_df = pd.merge(test_df, price_df, on=['item_id', 'month_of_year'], how='left')


## 월별 품목의 물량 추가(mean, max)

- 전달 물량 추가 / 3개월 물량 추가

In [ ]:
# item_month_supply = train_df.groupby(['item', 'month_of_year'])
item_month_supply = train_df.groupby(['item_id', 'month_of_year'])

agg = {'supply' : ['mean', 'max']}
supply_df = item_month_supply.agg(agg).reset_index()

In [ ]:
supply_df.columns = ['item_id', 'month_of_year', 'month_supply_mean', 'month_supply_max']

train_df = pd.merge(train_df, supply_df, on=['item_id', 'month_of_year'], how='left')
test_df = pd.merge(test_df, supply_df, on=['item_id', 'month_of_year'], how='left')


In [ ]:
# 공급과 가격의 상관관계
sup_price_corr = train_df['supply'].corr(train_df['price'])
sup_price_corr

-0.002296736600486043

## international 데이터 merge

In [ ]:
international = pd.read_csv(f"{DATA_PATH}international_trade.csv")

In [ ]:
international = international.rename(columns={
    '기간': 'period',
    '품목명': 'item',
    '수출 중량': 'exWeight',
    '수출 금액': 'exValue',
    '수입 중량': 'imWeight',
    '수입 금액': 'imValue',
    '무역수지': 'tradeBalance'
})

international.head()

# international['tradeBalance'] = international['exValue'] - international['imValue']

,period,item,exWeight,exValue,imWeight,imValue,tradeBalance
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38


In [ ]:
international['year'] = international['period'].apply(lambda x : int(x[0:4]))
international['month_ori'] = international['period'].apply(lambda x : int(x[5:7]))
international.drop(columns = 'period', inplace=True)
international

,item,exWeight,exValue,imWeight,imValue,tradeBalance,year,month_ori
0,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,2019,1
1,양파,821330,222,4003206,1118,-896,2019,1
2,쪽파,60,1,93405,128,-127,2019,1
3,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
4,방울다다기 양배추,0,0,7580,38,-38,2019,1
...,...,...,...,...,...,...,...,...
1269,포포(papaw)[파파야(papaya)],0,0,23830,71,-71,2023,2
1270,사과,135165,351,0,0,351,2023,2
1271,배,2206012,5411,1,0,5411,2023,2
1272,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0,2023,2


In [ ]:
selected_items = international[international['item'].str.contains('감귤|브로콜리|무|당근|양배추')].copy()
mask = (selected_items['item'] != '무화과') & (selected_items['item'] != '방울다다기 양배추')
selected_items = selected_items[mask]

In [ ]:
selected_items

,item,exWeight,exValue,imWeight,imValue,tradeBalance,year,month_ori
3,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
5,양배추,184650,94,395802,90,4,2019,1
8,당근,23150,22,7466150,2955,-2934,2019,1
17,감귤,58368,172,0,0,172,2019,1
28,꽃양배추와 브로콜리(broccoli),780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
1248,꽃양배추와 브로콜리(broccoli),24,0,332640,352,-352,2023,2
1250,양배추,13188,13,377456,104,-91,2023,2
1253,당근,22510,20,9260020,3758,-3737,2023,2
1254,순무,4000,4,2,0,4,2023,2


In [ ]:
selected_items['item'] = selected_items['item'].replace({
    '.*감귤.*': 'TG',
    '.*브로콜리.*': 'BC',
    '.*무.*': 'RD',
    '.*당근.*': 'CR',
    '.*양배추.*': 'CB'
}, regex = True)
selected_items

,item,exWeight,exValue,imWeight,imValue,tradeBalance,year,month_ori
3,BC,160,1,638913,563,-562,2019,1
5,CB,184650,94,395802,90,4,2019,1
8,CR,23150,22,7466150,2955,-2934,2019,1
17,TG,58368,172,0,0,172,2019,1
28,BC,780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
1248,BC,24,0,332640,352,-352,2023,2
1250,CB,13188,13,377456,104,-91,2023,2
1253,CR,22510,20,9260020,3758,-3737,2023,2
1254,RD,4000,4,2,0,4,2023,2


In [ ]:
train_df = train_df.merge(selected_items, on=['year', 'month_ori', 'item'], how='left')
test_df = test_df.merge(selected_items, on=['year', 'month_ori', 'item'], how='left')
train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,month_of_year,month_price_mean,month_price_max,month_supply_mean,month_supply_max,exWeight,exValue,imWeight,imValue,tradeBalance
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2019-01,1513.516129,2932.0,23069.290323,60601.0,58368.0,172.0,0.0,0.0,172.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2019-01,1513.516129,2932.0,23069.290323,60601.0,58368.0,172.0,0.0,0.0,172.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,...,2019-01,1513.516129,2932.0,23069.290323,60601.0,58368.0,172.0,0.0,0.0,172.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,...,2019-01,1513.516129,2932.0,23069.290323,60601.0,58368.0,172.0,0.0,0.0,172.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,...,2019-01,1513.516129,2932.0,23069.290323,60601.0,58368.0,172.0,0.0,0.0,172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,...,2023-02,434.857143,689.0,382965.714286,657220.0,4000.0,4.0,2.0,0.0,4.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,...,2023-02,434.857143,689.0,382965.714286,657220.0,4000.0,4.0,2.0,0.0,4.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,...,2023-03,542.000000,574.0,429240.000000,477220.0,NaN,NaN,NaN,NaN,NaN
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,...,2023-03,542.000000,574.0,429240.000000,477220.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_df['exValue'] = train_df['exValue']*1000
train_df['imValue'] = train_df['imValue']*1000
# train_df['tradeBalance'] = train_df['exValue'] - train_df['imValue']
train_df['exPrice'] = train_df['exValue'] / train_df['exWeight']
train_df['imPrice'] = train_df['imValue'] / train_df['imWeight']

test_df['exValue'] = test_df['exValue']*1000
test_df['imValue'] = test_df['imValue']*1000
# test_df['tradeBalance'] = test_df['exValue'] - test_df['imValue']
test_df['exPrice'] = test_df['exValue'] / test_df['exWeight']
test_df['imPrice'] = test_df['imValue'] / test_df['imWeight']


train_df['exWeight'] = np.log(train_df['exWeight'])
train_df['imWeight'] = np.log(train_df['imWeight'])
train_df['exValue'] = np.log(train_df['exValue'])
train_df['imValue'] = np.log(train_df['imValue'])
train_df['tradeBalance'] = np.log(train_df['tradeBalance'])

test_df['exWeight'] = np.log(test_df['exWeight'])
test_df['imWeight'] = np.log(test_df['imWeight'])
test_df['exValue'] = np.log(test_df['exValue'])
test_df['imValue'] = np.log(test_df['imValue'])
test_df['tradeBalance'] = np.log(test_df['tradeBalance'])

# test_df['tradeBalance'] = test_df['exValue'] - test_df['imValue']

# train_df

In [ ]:
# 2. category
international.loc[international['item'].str.contains('감귤|배|복숭아|사과|살구|수박|자두|참외'), 'category'] = 'ex_fruit'
international.loc[international['item'].str.contains('그레이프|망고|오렌지|파인애플|체리'), 'category'] = 'im_fruit'

international.loc[international['item'].str.contains('토마토|순무|(?!다다기,꽃)양배추|양파|오이'), 'category'] = 'ex_vege'
international.loc[international['item'].str.contains('상추|구아바|브로콜리|당근|레몬|무화과|방울다다기|샬롯|양파|치커리|쪽파|파파야'), 'category'] = 'im_vege'

# 완두, 콩, 대추야자  제외 (과일이나 채소 아닌 곡류)
international_type = international.groupby(['category','year','month_ori'])[['exWeight','exValue','imWeight','imValue','tradeBalance']].sum().reset_index()
international_type

,category,year,month_ori,exWeight,exValue,imWeight,imValue,tradeBalance
0,ex_fruit,2019,1,3635002,10297,1500,19,10277
1,ex_fruit,2019,2,1083795,2748,26380,257,2491
2,ex_fruit,2019,3,1170348,3188,22159,164,3024
3,ex_fruit,2019,4,544789,1406,13457,36,1370
4,ex_fruit,2019,5,397765,1122,0,0,1122
...,...,...,...,...,...,...,...,...
195,im_vege,2022,10,251568,288,11785495,7492,-7205
196,im_vege,2022,11,44266,53,8347745,7046,-6993
197,im_vege,2022,12,27937,48,13154816,8214,-8167
198,im_vege,2023,1,18866,45,9114212,6431,-6387


In [ ]:
train_df.loc[train_df['item'].str.contains('TG'), 'category'] = 'ex_fruit'
train_df.loc[train_df['item'].str.contains('CB|RD'), 'category'] = 'ex_vege'
train_df.loc[train_df['item'].str.contains('BC|CR'), 'category'] = 'im_vege'

train_df.loc[train_df['item'].str.contains('TG'), 'type'] = 1
train_df.loc[train_df['item'].str.contains('CB|RD|BC|CR'), 'type'] = 0

test_df.loc[test_df['item'].str.contains('TG'), 'category'] = 'ex_fruit'
test_df.loc[test_df['item'].str.contains('CB|RD'), 'category'] = 'ex_vege'
test_df.loc[test_df['item'].str.contains('BC|CR'), 'category'] = 'im_vege'

test_df.loc[test_df['item'].str.contains('TG'), 'type'] = 1
test_df.loc[test_df['item'].str.contains('CB|RD|BC|CR'), 'type'] = 0

In [ ]:
#

In [ ]:
train_df = train_df.merge(international_type, on=['category','year','month_ori'], how='left',suffixes = ('','_category') )
test_df = test_df.merge(international_type, on=['category','year','month_ori'], how='left',suffixes = ('','_category') )

train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,tradeBalance,exPrice,imPrice,category,type,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,3635002.0,10297.0,1500.0,19.0,10277.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,3635002.0,10297.0,1500.0,19.0,10277.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,3635002.0,10297.0,1500.0,19.0,10277.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,3635002.0,10297.0,1500.0,19.0,10277.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,3635002.0,10297.0,1500.0,19.0,10277.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,...,1.386294,1.00000,0.0,ex_vege,0.0,66336.0,219.0,377458.0,104.0,115.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,...,1.386294,1.00000,0.0,ex_vege,0.0,66336.0,219.0,377458.0,104.0,115.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,...,NaN,NaN,NaN,ex_vege,0.0,NaN,NaN,NaN,NaN,NaN
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,...,NaN,NaN,NaN,ex_vege,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_df['exWeight_category'] = np.log(train_df['exWeight_category'])
train_df['exValue_category'] = np.log(train_df['exValue_category'])
train_df['imWeight_category'] = np.log(train_df['imWeight_category'])
train_df['imValue_category'] = np.log(train_df['imValue_category'])
# train_df['tradeBalance_category'] = np.log(train_df['tradeBalance_category'])

test_df['exWeight_category'] = np.log(test_df['exWeight_category'])
test_df['exValue_category'] = np.log(test_df['exValue_category'])
test_df['imWeight_category'] = np.log(test_df['imWeight_category'])
test_df['imValue_category'] = np.log(test_df['imValue_category'])
# test_df['tradeBalance_category'] = np.log(test_df['tradeBalance_category'])




In [ ]:
train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,tradeBalance,exPrice,imPrice,category,type,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,...,5.147494,2.94682,NaN,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,...,1.386294,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,...,1.386294,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,...,NaN,NaN,NaN,ex_vege,0.0,NaN,NaN,NaN,NaN,NaN
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,...,NaN,NaN,NaN,ex_vege,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(0, 0)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59397 entries, 0 to 59396
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     59397 non-null  object        
 1   timestamp              59397 non-null  datetime64[ns]
 2   item                   59397 non-null  object        
 3   corporation            59397 non-null  object        
 4   location               59397 non-null  object        
 5   supply                 59397 non-null  float64       
 6   price                  59397 non-null  float64       
 7   item_id                59397 non-null  object        
 8   year                   59397 non-null  int64         
 9   month_ori              59397 non-null  int64         
 10  week                   59397 non-null  UInt32        
 11  day                    59397 non-null  int64         
 12  day_of_week            59397 non-null  int64         
 13  s

## 기타 피쳐추가

In [ ]:
# corp_balance_supply = train_df.groupby(['corporation','year','month_ori','category','item'])[['supply','tradeBalance','tradeBalance_category']].mean().reset_index()

In [ ]:
# train_df = train_df.merge(corp_balance_supply, on = ['corporation','year','month_ori','category','item'], how='left', suffixes = ('','_corp_balance') )
# test_df = test_df.merge(corp_balance_supply, on = ['corporation','year','month_ori','category','item'], how='left', suffixes = ('','_corp_balance') )

# test_df.rename(columns = {'supply' :'supply_corp_balance'}, inplace=True)

In [ ]:
# train_df['supply_corp_balance'] = np.log(train_df['supply_corp_balance'])
# test_df['supply_corp_balance'] = np.log(test_df['supply_corp_balance'])

# test_df

In [ ]:
# corp_balance_price = train_df.groupby(['corporation','year','month_ori','type','category','item'])[['price']].mean().reset_index()

In [ ]:
# train_df = train_df.merge(corp_balance_price, on = ['corporation','year','month_ori','type','category','item'], how='left', suffixes = ('','_corp_balance') )
# test_df = test_df.merge(corp_balance_price, on = ['corporation','year','month_ori','type','category','item'], how='left', suffixes = ('','_corp_balance') )

# test_df.rename(columns = {'price' :'price_corp_balance'}, inplace=True)

In [ ]:
train_df[train_df.item_id == 'TG_A_J']['price']


0          0.0
1          0.0
2       1728.0
3       1408.0
4       1250.0
         ...  
1518    2657.0
1519    3922.0
1520    3397.0
1521    3195.0
1522    3640.0
Name: price, Length: 1523, dtype: float64

In [ ]:
train_df['price_change_day1'] = (train_df['price'] - train_df['price'].shift(1)) / train_df['price'].shift(1)
train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,exPrice,imPrice,category,type,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category,price_change_day1
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,NaN
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,inf
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-0.185185
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-0.112216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,...,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,inf
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,...,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,0.134615
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,...,0.00000,0.0,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.080979
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,...,0.00000,0.0,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,-0.088850


In [ ]:
for item in train_df.item_id.unique() :
    tmp = train_df[train_df.item_id == item]
    previous_day_price = tmp['price'].shift(1)

    # Replace 0 or NaN values with a small non-zero value (0.000001)
    previous_day_price.replace({0: 1, pd.NaT: 1}, inplace=True)
    train_df.loc[train_df['item_id'] == item, 'price_change_day1'] = (tmp['price'] - previous_day_price) / previous_day_price

train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,exPrice,imPrice,category,type,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category,price_change_day1
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-1.000000
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,1727.000000
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-0.185185
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-0.112216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,...,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,467.000000
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,...,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,0.134615
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,...,0.00000,0.0,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.080979
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,...,0.00000,0.0,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,-0.088850


In [ ]:
# for item in train_df.item_id.unique() :
#     tmp = train_df[train_df.item_id == item]
#     previous_day_price = tmp['supply'].shift(1)

#     # Replace 0 or NaN values with a small non-zero value (0.000001)
#     previous_day_price.replace({0: 1, pd.NaT: 1}, inplace=True)
#     train_df.loc[train_df['item_id'] == item, 'supply_change_day1'] = (tmp['supply'] - previous_day_price) / previous_day_price

In [ ]:
train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,exPrice,imPrice,category,type,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category,price_change_day1
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-1.000000
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,1727.000000
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-0.185185
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,...,2.94682,0.0,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,-0.112216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,...,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,467.000000
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,...,1.00000,0.0,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,0.134615
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,...,0.00000,0.0,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.080979
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,...,0.00000,0.0,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,-0.088850


In [ ]:
value_change = train_df.groupby(['item_id','type','corporation','year','month_ori'])[['price_change_day1']].mean().reset_index()

In [ ]:
train_df = train_df.merge(value_change, on = ['item_id','type','corporation','year','month_ori'], how='left', suffixes = ('','_month_mean'))
test_df = test_df.merge(value_change, on = ['item_id','type','corporation','year','month_ori'], how='left', suffixes = ('','_month_mean'))

test_df = test_df.rename(columns = {'price_change_day1':'price_change_day1_month_mean'})

In [ ]:
q1 = train_df['price_change_day1_month_mean'].quantile(0.25)
q2 = train_df['price_change_day1_month_mean'].quantile(0.5)
q3 = train_df['price_change_day1_month_mean'].quantile(0.75)

# 새로운 칼럼 추가
train_df['price_change_quartile'] = pd.cut(train_df['price_change_day1_month_mean'], bins=[float('-inf'), q1, q2, q3, float('inf')], labels=[0, 1, 2, 3])

q1 = test_df['price_change_day1_month_mean'].quantile(0.25)
q2 = test_df['price_change_day1_month_mean'].quantile(0.5)
q3 = test_df['price_change_day1_month_mean'].quantile(0.75)

# 새로운 칼럼 추가
test_df['price_change_quartile'] = pd.cut(test_df['price_change_day1_month_mean'], bins=[float('-inf'), q1, q2, q3, float('inf')], labels=[0, 1, 2, 3])


In [ ]:
pd.DataFrame(train_df.price_change_day1_month_mean.value_counts().sort_index())

,price_change_day1_month_mean
-1.000000,21761
-0.973979,31
-0.971503,31
-0.967742,31
-0.297583,3
...,...
2224.486958,31
2357.099574,31
2384.791545,30
2430.132447,30


In [ ]:
train_df['price_mean_zero'] = (train_df['price_change_day1_month_mean'] == -1.0).astype(int)
test_df['price_mean_zero'] = (test_df['price_change_day1_month_mean'] == -1.0).astype(int)

In [ ]:
test_df.price_change_day1_month_mean.value_counts()

-1.000000      224
-0.018015       28
 0.078774       28
-0.013226       28
-0.021350       28
-0.037550       28
 0.025969       28
 0.044896       28
-0.048134       28
-0.023607       28
-0.046007       28
-0.297583       28
 204.693723     28
 0.017551       28
 178.398510     28
 0.101078       28
-0.009773       28
 0.007039       28
 226.413413     28
 0.125283       28
-0.061582       28
-0.054883       28
-0.051090       28
 0.247559       28
 0.080228       28
 633.950630     28
 0.109076       28
 0.039071       28
 0.259830       28
 0.035849       28
 0.188203       28
 0.001200       28
Name: price_change_day1_month_mean, dtype: int64

In [ ]:
cols_drop = ['price_change_day1']
train_df.drop(columns = cols_drop, inplace=True)
train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,category,type,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category,price_change_day1_month_mean,price_change_quartile,price_mean_zero
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,332.064205,2,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,...,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,332.064205,2,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,...,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,332.064205,2,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,...,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,332.064205,2,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,...,ex_fruit,1.0,15.106120,9.239608,7.313220,2.944439,10277.0,332.064205,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,...,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,70.215195,1,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,...,ex_vege,0.0,11.102488,5.389072,12.841215,4.644391,115.0,70.215195,1,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,...,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.001200,1,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,...,ex_vege,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.001200,1,0


# Encoding

## 원핫

In [ ]:
# # item OneHotEncoding
# train_df = pd.get_dummies(train_df, columns=['item'])
# test_df = pd.get_dummies(test_df, columns=['item'])

## 라벨

In [ ]:
obj_cols = ['item', 'corporation', 'location','category']

for col in obj_cols:
    enc = LabelEncoder()
    train_df[col]=enc.fit_transform(train_df[col])
    test_df[col]=enc.transform(test_df[col])

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,category,type,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category,price_change_day1_month_mean,price_change_quartile,price_mean_zero
0,TG_A_J_20190101,2019-01-01,4,0,0,0.0,0.0,TG_A_J,2019,1,...,0,1.0,15.10612,9.239608,7.31322,2.944439,10277.0,332.064205,2,0
1,TG_A_J_20190102,2019-01-02,4,0,0,0.0,0.0,TG_A_J,2019,1,...,0,1.0,15.10612,9.239608,7.31322,2.944439,10277.0,332.064205,2,0
2,TG_A_J_20190103,2019-01-03,4,0,0,60601.0,1728.0,TG_A_J,2019,1,...,0,1.0,15.10612,9.239608,7.31322,2.944439,10277.0,332.064205,2,0
3,TG_A_J_20190104,2019-01-04,4,0,0,25000.0,1408.0,TG_A_J,2019,1,...,0,1.0,15.10612,9.239608,7.31322,2.944439,10277.0,332.064205,2,0
4,TG_A_J_20190105,2019-01-05,4,0,0,32352.0,1250.0,TG_A_J,2019,1,...,0,1.0,15.10612,9.239608,7.31322,2.944439,10277.0,332.064205,2,0


In [ ]:
train_df['week'] = train_df['week'].astype(float)
train_df['cos_week_of_year'] = train_df['cos_week_of_year'].astype(float)
train_df['sin_week_of_year'] = train_df['sin_week_of_year'].astype(float)

test_df['week'] = test_df['week'].astype(float)
test_df['cos_week_of_year'] = test_df['cos_week_of_year'].astype(float)
test_df['sin_week_of_year'] = test_df['sin_week_of_year'].astype(float)

## 카테고리화

In [ ]:
# 10분위수로 카테고리화 하기

# 10분위수로 나누기

train_df['price_category'] = pd.qcut(train_df['price'], q=10, labels=False, duplicates='drop')
train_df['supply_category'] = pd.qcut(train_df['supply'], q=10, labels=False, duplicates='drop')


# 각 구간의 평균값 계산
mean_values = train_df.groupby('price_category')['price'].mean()
train_df['price_category_mean'] = train_df['price_category'].map(mean_values)

# 각 구간의 평균값 계산
mean_values = train_df.groupby('supply_category')['supply'].mean()
train_df['supply_category_mean'] = train_df['supply_category'].map(mean_values)


In [ ]:
# 훈련 데이터에서 각 조합에서의 최빈 구간을 찾아서 매핑
item_price_mode = train_df.groupby(['item', 'month_ori','day_of_week'])['price_category'].apply(lambda x: x.mode().iloc[0])
item_supply_mode = train_df.groupby(['item','month_ori','day_of_week'])['supply_category'].apply(lambda x: x.mode().iloc[0])

test_df['price_category'] = test_df.apply(lambda row: item_price_mode.get((row['item'], row['month_ori'], row['day_of_week']), None), axis=1)
test_df['supply_category'] = test_df.apply(lambda row: item_supply_mode.get((row['item'], row['month_ori'], row['day_of_week']), None), axis=1)

mean_values_price = train_df.groupby('price_category')['price'].mean()
mean_values_supply = train_df.groupby('supply_category')['supply'].mean()

# 테스트 데이터에 평균값 추가
test_df['price_category_mean'] = test_df['price_category'].map(mean_values_price)
test_df['supply_category_mean'] = test_df['supply_category'].map(mean_values_supply)


In [ ]:
train_df['price_category_mean'] = np.log(train_df['price_category_mean'])
train_df['supply_category_mean'] = np.log(train_df['supply_category_mean'])

test_df['price_category_mean'] = np.log(test_df['price_category_mean'])
test_df['supply_category_mean'] = np.log(test_df['supply_category_mean'])

# AutoGluon

In [ ]:
train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,imWeight_category,imValue_category,tradeBalance_category,price_change_day1_month_mean,price_change_quartile,price_mean_zero,price_category,supply_category,price_category_mean,supply_category_mean
0,TG_A_J_20190101,2019-01-01,4,0,0,0.0,0.0,TG_A_J,2019,1,...,7.313220,2.944439,10277.0,332.064205,2,0,0,0,0.153236,-1.686708
1,TG_A_J_20190102,2019-01-02,4,0,0,0.0,0.0,TG_A_J,2019,1,...,7.313220,2.944439,10277.0,332.064205,2,0,0,0,0.153236,-1.686708
2,TG_A_J_20190103,2019-01-03,4,0,0,60601.0,1728.0,TG_A_J,2019,1,...,7.313220,2.944439,10277.0,332.064205,2,0,2,4,7.331624,11.520248
3,TG_A_J_20190104,2019-01-04,4,0,0,25000.0,1408.0,TG_A_J,2019,1,...,7.313220,2.944439,10277.0,332.064205,2,0,2,4,7.331624,11.520248
4,TG_A_J_20190105,2019-01-05,4,0,0,32352.0,1250.0,TG_A_J,2019,1,...,7.313220,2.944439,10277.0,332.064205,2,0,2,4,7.331624,11.520248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,3,5,0,452440.0,468.0,RD_F_J,2023,2,...,12.841215,4.644391,115.0,70.215195,1,0,1,4,6.343372,11.520248
59393,RD_F_J_20230228,2023-02-28,3,5,0,421980.0,531.0,RD_F_J,2023,2,...,12.841215,4.644391,115.0,70.215195,1,0,1,4,6.343372,11.520248
59394,RD_F_J_20230301,2023-03-01,3,5,0,382980.0,574.0,RD_F_J,2023,3,...,0.000000,0.000000,0.0,0.001200,1,0,1,4,6.343372,11.520248
59395,RD_F_J_20230302,2023-03-02,3,5,0,477220.0,523.0,RD_F_J,2023,3,...,0.000000,0.000000,0.0,0.001200,1,0,1,4,6.343372,11.520248


In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [ ]:
train_dropcols = ['ID', 'supply', 'month_of_year']
test_dropcols = ['ID', 'month_of_year']

In [ ]:
# train_df_tg = train_df_tg.drop(columns = train_dropcols)
# test_df_tg = test_df_tg.drop(columns = test_dropcols)

# train_df_bc = train_df_bc.drop(columns = train_dropcols)
# test_df_bc = test_df_bc.drop(columns = test_dropcols)

# train_df_rd = train_df_rd.drop(columns = train_dropcols)
# test_df_rd = test_df_rd.drop(columns = test_dropcols)

# train_df_cr = train_df_cr.drop(columns = train_dropcols)
# test_df_cr = test_df_cr.drop(columns = test_dropcols)

# train_df_cb = train_df_cb.drop(columns = train_dropcols)
# test_df_cb = test_df_cb.drop(columns = test_dropcols)

In [ ]:
train_ft = train_df.drop(columns = train_dropcols)
test_ft = test_df.drop(columns = test_dropcols)

In [ ]:
train_ft

,timestamp,item,corporation,location,price,item_id,year,month_ori,week,day,...,imWeight_category,imValue_category,tradeBalance_category,price_change_day1_month_mean,price_change_quartile,price_mean_zero,price_category,supply_category,price_category_mean,supply_category_mean
0,2019-01-01,4,0,0,0.0,TG_A_J,2019,1,1.0,1,...,7.313220,2.944439,10277.0,332.064205,2,0,0,0,0.153236,-1.686708
1,2019-01-02,4,0,0,0.0,TG_A_J,2019,1,1.0,2,...,7.313220,2.944439,10277.0,332.064205,2,0,0,0,0.153236,-1.686708
2,2019-01-03,4,0,0,1728.0,TG_A_J,2019,1,1.0,3,...,7.313220,2.944439,10277.0,332.064205,2,0,2,4,7.331624,11.520248
3,2019-01-04,4,0,0,1408.0,TG_A_J,2019,1,1.0,4,...,7.313220,2.944439,10277.0,332.064205,2,0,2,4,7.331624,11.520248
4,2019-01-05,4,0,0,1250.0,TG_A_J,2019,1,1.0,5,...,7.313220,2.944439,10277.0,332.064205,2,0,2,4,7.331624,11.520248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,2023-02-27,3,5,0,468.0,RD_F_J,2023,2,9.0,27,...,12.841215,4.644391,115.0,70.215195,1,0,1,4,6.343372,11.520248
59393,2023-02-28,3,5,0,531.0,RD_F_J,2023,2,9.0,28,...,12.841215,4.644391,115.0,70.215195,1,0,1,4,6.343372,11.520248
59394,2023-03-01,3,5,0,574.0,RD_F_J,2023,3,9.0,1,...,0.000000,0.000000,0.0,0.001200,1,0,1,4,6.343372,11.520248
59395,2023-03-02,3,5,0,523.0,RD_F_J,2023,3,9.0,2,...,0.000000,0.000000,0.0,0.001200,1,0,1,4,6.343372,11.520248


In [ ]:
train_ft.columns

Index(['timestamp', 'item', 'corporation', 'location', 'price', 'item_id',
       'year', 'month_ori', 'week', 'day', 'day_of_week', 'sin_week',
       'cos_week', 'sin_dayofweek', 'cos_dayofweek', 'holiday', 'cos_year',
       'sin_year', 'cos_dow', 'sin_dow', 'cos_week_of_year',
       'sin_week_of_year', 'month_price_mean', 'month_price_max',
       'month_supply_mean', 'month_supply_max', 'exWeight', 'exValue',
       'imWeight', 'imValue', 'tradeBalance', 'exPrice', 'imPrice', 'category',
       'type', 'exWeight_category', 'exValue_category', 'imWeight_category',
       'imValue_category', 'tradeBalance_category',
       'price_change_day1_month_mean', 'price_change_quartile',
       'price_mean_zero', 'price_category', 'supply_category',
       'price_category_mean', 'supply_category_mean'],
      dtype='object')

In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.1 MB/

In [ ]:
# from autogluon.tabular import TabularPredictor
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
data = TimeSeriesDataFrame(train_ft)

In [ ]:
data.static_features

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price",
    eval_metric="RMSE",

    # quantile_levels=[0.05, 0.5, 0.95],
    # known_covariates_names=["holiday"]

)
# seed 고정
# predictor.fit(data,presets="best_quality",random_seed=SEED)
predictor.fit(data,random_seed=SEED, presets = 'best_quality',time_limit=3600)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: best_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': {'num_trials': 3,
                                'scheduler': 'local',
                                'searcher': 'auto'},
 'hyperparameters': 'best_quality',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price',
 'time_limit': 3600,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231115_133705/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be mult

  0%|          | 0/3 [00:00<?, ?it/s]

	Trained 2 models while tuning DeepAR.
	-701.3304     = Validation score (-RMSE)
	261.31  s     = Total tuning time
Hyperparameter tuning model: TemporalFusionTransformer. Tuning model for up to 359.98s of the 3599.84s remaining.
	nan           = Validation score (-RMSE)
	360.80  s     = Training runtime
	0.40    s     = Validation (prediction) runtime
Hyperparameter tuning model: PatchTST. Tuning model for up to 359.98s of the 3599.84s remaining.
	-1330.5139    = Validation score (-RMSE)
	124.06  s     = Training runtime
	0.34    s     = Validation (prediction) runtime
Hyperparameter tuning model: DirectTabular. Tuning model for up to 359.98s of the 3599.84s remaining.
	-972.7877     = Validation score (-RMSE)
	88.71   s     = Training runtime
	0.58    s     = Validation (prediction) runtime
Hyperparameter tuning model: AutoARIMA. Tuning model for up to 359.98s of the 3599.84s remaining.
	-850.8046     = Validation score (-RMSE)
	0.10    s     = Training runtime
	340.51  s     = Valid

In [ ]:
train_ft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59397 entries, 0 to 59396
Data columns (total 47 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     59397 non-null  datetime64[ns]
 1   item                          59397 non-null  int64         
 2   corporation                   59397 non-null  int64         
 3   location                      59397 non-null  int64         
 4   price                         59397 non-null  float64       
 5   item_id                       59397 non-null  object        
 6   year                          59397 non-null  int64         
 7   month_ori                     59397 non-null  int64         
 8   week                          59397 non-null  float64       
 9   day                           59397 non-null  int64         
 10  day_of_week                   59397 non-null  int64         
 11  sin_week                    

# 예측

In [ ]:
# 모델 재학습
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	62.32   s     = Training runtime
Fitting model: DeepAR/T1_FULL | Skipping fit via cloning parent ...
Fitting model: DeepAR/T2_FULL | Skipping fit via cloning parent ...
Fitting model: TemporalFusionTransformer_FULL | Skipping fit via cloning parent ...
Fitting model: PatchTST_FULL | Skipping fit via cloning parent ...
Fitting model: DirectTabular_FULL
	115.33  s     = Training runtime
Fitting model: AutoARIMA_FULL |

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR/T1': 'DeepAR/T1_FULL',
 'DeepAR/T2': 'DeepAR/T2_FULL',
 'TemporalFusionTransformer': 'TemporalFusionTransformer_FULL',
 'PatchTST': 'PatchTST_FULL',
 'DirectTabular': 'DirectTabular_FULL',
 'AutoARIMA': 'AutoARIMA_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [ ]:
pred = predictor.predict(data, random_seed=SEED)
pred

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3242.071525  1549.218227  2162.892889  2545.286877   
        2023-03-05   354.808082  -878.216911  -455.567545  -150.553661   
        2023-03-06  3580.438274  1017.793540  1810.555716  2349.412580   
        2023-03-07  3372.010801   943.870165  1808.921343  2437.812396   
        2023-03-08  3352.047592   667.485528  1675.526018  2334.746216   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   521.511787    78.543621   237.821651   343.468321   
        2023-03-28   524.768013    14.116873   256.036669   360.441730   
        2023-03-29   518.897553     6.084375   230.540536   361.468896   
        2023-03-30   505.452293    -1.686151   223.567949   347.163034   
        2023-03-31   549.246238    60.236539   224.481742   348.481108   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2895.013867  3172.365958  3505.772981  3860.110079   
        2023-03-05   109.222188   352.059564   594.523043   853.860893   
        2023-03-06  2783.766409  3222.135783  3654.346467  4150.529641   
        2023-03-07  2951.241837  3395.803770  3881.711894  4407.237530   
        2023-03-08  2874.705026  3383.232163  3865.295920  4390.282077   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   428.600468   501.979316   586.237651   683.313890   
        2023-03-28   448.824935   525.647072   615.187165   711.068631   
        2023-03-29   442.779173   525.487488   610.231356   708.562964   
        2023-03-30   441.268279   521.182243   612.366562   703.246038   
        2023-03-31   448.972381   536.927116   621.897628   721.175614   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4282.345497  4919.387347  
        2023-03-05  1158.740153  1582.730758  
        2023-03-06  4676.410306  5464.642938  
        2023-03-07  4967.419107  5921.585397  
        2023-03-08  5061.855299  6202.835242  
...                         ...          ...  
RD_F_J  2023-03-27   800.889393  1019.079014  
        2023-03-28   815.443419   993.283168  
        2023-03-29   823.632011  1001.181693  
        2023-03-30   805.399136  1003.404518  
        2023-03-31   830.779441  1014.723649  

[1092 rows x 10 columns]

In [ ]:
# predictor.leaderboard(data, silent=True)


In [ ]:
predictor.get_model_best()

'WeightedEnsemble_FULL'

In [ ]:
pred = pred.reset_index()['mean']
pred

0       3242.071525
1        354.808082
2       3580.438274
3       3372.010801
4       3352.047592
           ...     
1087     521.511787
1088     524.768013
1089     518.897553
1090     505.452293
1091     549.246238
Name: mean, Length: 1092, dtype: float64

In [ ]:

# 마이너스 값 0으로 치환
pred[pred < 0] = 0

# 제출

In [ ]:
submission

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [ ]:
submission['answer'] = pred
submission

,ID,answer
0,TG_A_J_20230304,3242.071525
1,TG_A_J_20230305,354.808082
2,TG_A_J_20230306,3580.438274
3,TG_A_J_20230307,3372.010801
4,TG_A_J_20230308,3352.047592
...,...,...
1087,RD_F_J_20230327,521.511787
1088,RD_F_J_20230328,524.768013
1089,RD_F_J_20230329,518.897553
1090,RD_F_J_20230330,505.452293


In [ ]:
submission.to_csv(f"{DATA_PATH}submission_v12_1115.csv", index=False)
submission.to_csv("submission_v12_1115.csv", index=False)